In [ ]:
pip install scikit-learn deap numpy

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from deap import base, creator, tools, algorithms

In [ ]:
# Assuming you have a dataset X containing features and y containing labels
# Replace X and y with your actual dataset
X = np.random.rand(100, 10)  # Example feature matrix
y = np.random.randint(0, 2, size=100)  # Example labels (binary classification)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define the genetic algorithm parameters
NUM_FEATURES = X.shape[1]
POPULATION_SIZE = 50
NUM_GENERATIONS = 10

In [ ]:
# Define evaluation function for genetic algorithm
def evaluate(individual):
    selected_features = [index for index, value in enumerate(individual) if value == 1]
    if len(selected_features) == 0:
        return 0,  # Avoid choosing zero features
    X_train_selected = X_train_scaled[:, selected_features]
    X_test_selected = X_test_scaled[:, selected_features]
    clf = SVC(kernel='linear')
    clf.fit(X_train_selected, y_train)
    y_pred = clf.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,

In [ ]:
# Create genetic algorithm toolbox
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.choice, [0, 1])  # Gene representing feature presence
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=NUM_FEATURES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
# Run genetic algorithm
population = toolbox.population(n=POPULATION_SIZE)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=NUM_GENERATIONS, verbose=True)

# Get the best individual (i.e., chromosome) from the final population
best_individual = tools.selBest(population, k=1)[0]
selected_features = [index for index, value in enumerate(best_individual) if value == 1]

gen	nevals
0  	50    
1  	31    
2  	30    
3  	27    
4  	25    
5  	29    
6  	30    
7  	26    
8  	27    
9  	30    
10 	39    


In [ ]:
# Train SVM classifier using the selected features
X_train_selected = X_train_scaled[:, selected_features]
X_test_selected = X_test_scaled[:, selected_features]
clf = SVC(kernel='linear')
clf.fit(X_train_selected, y_train)

SVC(kernel='linear')

In [ ]:
# Evaluate the classifier on the test set
y_pred = clf.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.55
